In [17]:
import requests

url_api = "http://127.0.0.1:5000/"
end_point = "nlp_model/resources/classify"

params = {'workspace_id': 'dc1e7b3d-9137-4a20-a99c-d0d2029ef170',
          'sentence': ""}

def classify(sentence):
    params["sentence"] = sentence
    r = requests.get(url_api+end_point, params=params)
    return r.json()

In [18]:
classify("Como Saber valor locação veículo?")

{'entities': [],
 'intents': [{'confidence': 0.8773902164258855,
   'intent': 'Verificar_Valor_Locacao_Veiculo'},
  {'confidence': 0.8153024676052072, 'intent': 'Transferencia_Veiculo'},
  {'confidence': 0.7755817454152192, 'intent': 'Veiculo_Disponiveis'},
  {'confidence': 0.7543260721947231, 'intent': 'Devolucao_Veiculo'},
  {'confidence': 0.7480607797322354, 'intent': 'Solicitar_Troca_Veiculo'},
  {'confidence': 0.7413660687334673, 'intent': 'TrocaDevolucao_Ticketlog'},
  {'confidence': 0.7371386543654567,
   'intent': 'Verificar_Historico_Previdencia'},
  {'confidence': 0.7325612626595803, 'intent': 'Veiculo_Provisorio'},
  {'confidence': 0.7198557346679705, 'intent': 'Duvidas_Benefícios'},
  {'confidence': 0.718204650265764,
   'intent': 'Despesa_Pessoal_Marcado_Relatorio_Concur'}]}

# Upload Data

In [11]:
import json

file = open('/home/cloves/Downloads/skill-skill-for-comparation-(do-not-delete).json', "r") 
workspace = json.load(file)
workspace["intents"][-1]

{'intent': 'Verificar_Valor_Locacao_Veiculo',
 'examples': [{'text': 'Como faço para abrir um orçamento de locação?'},
  {'text': 'Como Saber valor locação veículo?'},
  {'text': 'Quem devo contatar referente a valores de locação de veículo?'},
  {'text': 'Quem pode realizar a locação de veículo?'}]}

In [12]:
end_point = "nlp_model/intents/"

for intent in intents_8:
    r = requests.post(url_api+end_point, params=params, json=intent)
    r = r.json()
    if r["result"] == "created":
        print("Intent %s was created with id %s"%(intent["intent"],r["_id"]))
    else:
        print("Intent %s was not created"%intent["intent"])

Intent Acessar_Concur was created with id SdxiYncB_pG9ugNATjOn
Intent Acessar_Painel_Colaborador was created with id UtxiYncB_pG9ugNAUzPy
Intent Adiantamento_e_QuandoReceber_13_Salario was created with id U9xiYncB_pG9ugNAWTMQ
Intent Agendamento_Alteracao_Ferias was created with id VNxiYncB_pG9ugNAXjMi
Intent Ajustar_Ponto was created with id VdxiYncB_pG9ugNAYzNG
Intent Ajuste_Informacao_Convenio was created with id V9xiYncB_pG9ugNAaDNe
Intent Alterar_Aprovador was created with id ddxiYncB_pG9ugNAbTN9
Intent Alterar_Conta_Pagamento was created with id OtxiYncB_pG9ugNAcjSi
Intent Alterar_EnquadramentoMerito was created with id PdxiYncB_pG9ugNAdzTJ
Intent Alterar_Vale_RefeicaoAlimentacao was created with id RdxiYncB_pG9ugNAfDTw
Intent Bem_Estar was created with id RtxiYncB_pG9ugNAgTT-
Intent Bot_Capabilities was created with id R9xiYncB_pG9ugNAhzQW
Intent Cartao_Clonado_Cambial was created with id SdxiYncB_pG9ugNAjDQy
Intent Contracheque_Recebido_Sem_Constar_Conta was created with id S9xi

# Function to talk with Watson Model

In [14]:
import json
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

API_KEY = "nA36HqlqQkQSDf0EywIeYGPl0qI_nNvLqvBUI_zhhUmR" # os.environ['API_KEY']
SERVICE_URL = "https://api.us-south.assistant.watson.cloud.ibm.com/instances/12c017ac-1bb4-4364-8493-9cc9ba98aa26" # os.environ['SERVICE_URL']
ASSISTANT_ID = "11cdca6b-e73e-48fc-8523-13d9ee68df8b" # os.environ['ASSISTANT_ID']

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV1(
    version='2020-04-01',
    authenticator = authenticator
)

assistant.set_service_url(SERVICE_URL)

def watson_predict(sentence):
    response = assistant.message(
        workspace_id=ASSISTANT_ID,
        input={
            'message_type': 'text',
            'text': sentence
        }
    ).get_result()
    if response.get("intents") and response["intents"]:
        return (response["intents"][0]["intent"], response["intents"][0]["confidence"])
    else:
        return "failed to identify", -1.0

# Battle Omni NLP API BM25 vs. Watson Assistent

In [3]:
import pandas as pd

df_test = pd.read_csv("data/test_data.csv", sep="|")
df_test.head()

,example,intent
0,quais os assuntos você trata,Bot_Capabilities
1,"preciso trocar uma peça do meu carro, como dev...",Veiculo_Problema
2,"Como modificar a senha do cartão Alelo VR, VA?",Esquecer_Senha_Vale_RefeicaoAlimentacao
3,"Estou com o contra-cheque, mas o pagamento ain...",Contracheque_Recebido_Sem_Constar_Conta
4,"o pont fechou e o sistema está bloqueado, como...",Solicitar_Ponto_Pos_FechamentoReembolso


In [20]:
omni_right_answer = 0
watson_right_answer = 0
for _, row in df_test.iterrows():
    omni_result = classify(row.example)
    watson_result = watson_predict(row.example)
    
    print("-------------------------------------------")
    print("sentence: %s"%row.example)
    print("\n")
    print("Omni NLP API\n")
    print("predict_intent: %s"%omni_result["intents"][0]["intent"])
    print("score: %.2f"%omni_result["intents"][0]["confidence"])
    print("right_intent: %s"%row.intent)
    if omni_result["intents"][0]["intent"] == row.intent:
        omni_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %omni_right_answer)
    print("\n")
    
    print("Watson Assistent\n")
    print("predict_intent: %s"%watson_result[0])
    print("score: %.2f"%watson_result[1])
    print("right_intent: %s"%row.intent)
    if watson_result[0] == row.intent:
        watson_right_answer += 1
        print("is it right?: yes")
    else:
         print("is it right?: no")
    print("quantity of right answers: %s" %watson_right_answer)
    print("\n")

print("-------------------------------------------\n")
print("Total of questions: %d\n"%df_test.shape[0])
print("Omni NLP API\n") 
print("Total of right answers: %d"%omni_right_answer)
omni_acc = round(omni_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %.2f\n"%omni_acc)

print("Watson Assistent\n") 
print("Total of right answers: %d"%watson_right_answer)
watson_acc = round(watson_right_answer/df_test.shape[0], 2)*100
print("Accuracy: %.2f"%watson_acc)

-------------------------------------------
sentence: quais os assuntos você trata


Omni NLP API

predict_intent: Procedimentos_Convenio
score: 0.79
right_intent: Bot_Capabilities
is it right?: no
quantity of right answers: 0


Watson Assistent

predict_intent: General_Positive_Feedback
score: 0.43
right_intent: Bot_Capabilities
is it right?: no
quantity of right answers: 0


-------------------------------------------
sentence: preciso trocar uma peça do meu carro, como devo proceder?


Omni NLP API

predict_intent: Veiculo_Problema
score: 0.92
right_intent: Veiculo_Problema
is it right?: yes
quantity of right answers: 1


Watson Assistent

predict_intent: Veiculo_Problema
score: 0.46
right_intent: Veiculo_Problema
is it right?: yes
quantity of right answers: 1


-------------------------------------------
sentence: Como modificar a senha do cartão Alelo VR, VA?


Omni NLP API

predict_intent: Esquecer_Senha_Vale_RefeicaoAlimentacao
score: 0.94
right_intent: Esquecer_Senha_Vale_Refei

-------------------------------------------
sentence: Qual a fundação do CoE?


Omni NLP API

predict_intent: Foco_Cliente
score: 0.84
right_intent: Fundacao_Valores
is it right?: no
quantity of right answers: 15


Watson Assistent

predict_intent: Foco_Cliente
score: 0.49
right_intent: Fundacao_Valores
is it right?: no
quantity of right answers: 11


-------------------------------------------
sentence: Adotei recentemente meu filho e preciso colocar ele no meu plano odontologico


Omni NLP API

predict_intent: Incluir_Dependente_Plano_Saude
score: 0.91
right_intent: Incluir_Dependente_Plano_Saude
is it right?: yes
quantity of right answers: 16


Watson Assistent

predict_intent: Ajuste_Informacao_Convenio
score: 0.37
right_intent: Incluir_Dependente_Plano_Saude
is it right?: no
quantity of right answers: 11


-------------------------------------------
sentence: Não estou cadastrada, e minha matricula aparece inválida, o que fazer?


Omni NLP API

predict_intent: Usuario_Invalido_Nao

-------------------------------------------
sentence: é possível tirar apenas @sys-number de férias e vender os demais dias?


Omni NLP API

predict_intent: Divisao_Ferias
score: 0.94
right_intent: Divisao_Ferias
is it right?: yes
quantity of right answers: 30


Watson Assistent

predict_intent: Agendamento_Alteracao_Ferias
score: 0.62
right_intent: Divisao_Ferias
is it right?: no
quantity of right answers: 21


-------------------------------------------
sentence: Gostaria de repassar os colaboradores que estão na minha gestão no PontoMais para outro gestor, por um periodo.


Omni NLP API

predict_intent: Repassar_Gestao_Colaborador
score: 0.96
right_intent: Repassar_Gestao_Colaborador
is it right?: yes
quantity of right answers: 31


Watson Assistent

predict_intent: Repassar_Gestao_Colaborador
score: 0.65
right_intent: Repassar_Gestao_Colaborador
is it right?: yes
quantity of right answers: 22


-------------------------------------------
sentence: Quais são suas habilidades


Omni 

-------------------------------------------
sentence: Colaborador necessita do número da Unimed do seu dependente, o mesmo não conseguiu pelo app da Unimed e 0800 do COE.


Omni NLP API

predict_intent: Verificar_Numero_Carteirinha_Convenio
score: 0.95
right_intent: Verificar_Numero_Carteirinha_Convenio
is it right?: yes
quantity of right answers: 45


Watson Assistent

predict_intent: General_Negative_Feedback
score: 0.56
right_intent: Verificar_Numero_Carteirinha_Convenio
is it right?: no
quantity of right answers: 31


-------------------------------------------
sentence: você está bem?


Omni NLP API

predict_intent: Procedimento_Recusado_Convenio
score: 0.37
right_intent: Bem_Estar
is it right?: no
quantity of right answers: 45


Watson Assistent

predict_intent: Bem_Estar
score: 0.77
right_intent: Bem_Estar
is it right?: yes
quantity of right answers: 32


-------------------------------------------
sentence: Preciso do reembolso do meu relatório de despesas que está no Concur no

-------------------------------------------
sentence: O que fazer caso meu usuário não esteja cadastrado ou apresente inválido?


Omni NLP API

predict_intent: Ajustar_Ponto
score: 0.88
right_intent: Usuario_Invalido_NaoCadastrado
is it right?: no
quantity of right answers: 61


Watson Assistent

predict_intent: Incluir_Dependente_Plano_Saude
score: 0.33
right_intent: Usuario_Invalido_NaoCadastrado
is it right?: no
quantity of right answers: 39


-------------------------------------------
sentence: Como faço para prestar contas da devolução de um adiantamento no Concur?


Omni NLP API

predict_intent: Despesa_Pessoal_Marcado_Relatorio_Concur
score: 0.93
right_intent: Despesa_Pessoal_Marcado_Relatorio_Concur
is it right?: yes
quantity of right answers: 62


Watson Assistent

predict_intent: Procedimentos_Concur
score: 0.87
right_intent: Despesa_Pessoal_Marcado_Relatorio_Concur
is it right?: no
quantity of right answers: 39


-------------------------------------------
sentence: Olá, a 

...

-------------------------------------------
sentence: Estou com a prestação de contas do meu cartão de credito corporativo relativo ao ano de 2019 em aberto. Não consegui fazer a prestação de nenhum mês. Preciso de ajuda, quem pode me ajuda?


Omni NLP API

predict_intent: Procedimentos_Concur
score: 0.97
right_intent: Procedimentos_Concur
is it right?: yes
quantity of right answers: 125


Watson Assistent

predict_intent: Cartao_Clonado_Cambial
score: 0.39
right_intent: Procedimentos_Concur
is it right?: no
quantity of right answers: 86


-------------------------------------------
sentence: Só consigo ver numero cartao Dental.. é o mesmo numero do de saude? caso nao, como consigo o de saude - ja tentei no aplicativo e nao localizo.


Omni NLP API

predict_intent: Verificar_Numero_Carteirinha_Convenio
score: 0.95
right_intent: Verificar_Numero_Carteirinha_Convenio
is it right?: yes
quantity of right answers: 126


Watson Assistent

predict_intent: Download_Aplicativo_Convenio
score: 0

-------------------------------------------
sentence: Como solicito um ajuste de ponto pós fechamento ou reembolso?


Omni NLP API

predict_intent: Solicitar_Ponto_Pos_FechamentoReembolso
score: 0.94
right_intent: Solicitar_Ponto_Pos_FechamentoReembolso
is it right?: yes
quantity of right answers: 139


Watson Assistent

predict_intent: Procedimentos_Concur
score: 0.56
right_intent: Solicitar_Ponto_Pos_FechamentoReembolso
is it right?: no
quantity of right answers: 99


-------------------------------------------
sentence: O colaborador nao esta aparecendo na gestao do ponto do gestor.


Omni NLP API

predict_intent: IncluirRemover_Colaborador
score: 0.94
right_intent: Ponto_Colaborar_Não_Aparece
is it right?: no
quantity of right answers: 139


Watson Assistent

predict_intent: IncluirRemover_Colaborador
score: 0.68
right_intent: Ponto_Colaborar_Não_Aparece
is it right?: no
quantity of right answers: 99


-------------------------------------------
sentence: Recebi um adiantamento em 

-------------------------------------------
sentence: Como realizar a devolução do veículo?


Omni NLP API

predict_intent: Transferencia_Veiculo
score: 0.87
right_intent: Devolucao_Veiculo
is it right?: no
quantity of right answers: 154


Watson Assistent

predict_intent: Devolucao_Veiculo
score: 0.92
right_intent: Devolucao_Veiculo
is it right?: yes
quantity of right answers: 108


-------------------------------------------
sentence: Me envolvi em um acidente com o meu carro


Omni NLP API

predict_intent: Veiculo_Problema
score: 0.90
right_intent: Veiculo_Problema
is it right?: yes
quantity of right answers: 155


Watson Assistent

predict_intent: Veiculo_Problema
score: 0.38
right_intent: Veiculo_Problema
is it right?: yes
quantity of right answers: 109


-------------------------------------------
sentence: Ficarei ausente em virtude de viagem, como será feita gestão do ponto no período?


Omni NLP API

predict_intent: Repassar_Gestao_Colaborador
score: 0.92
right_intent: Repassa

-------------------------------------------
sentence: Como realizar um ajuste de ponto em caso de esquecimento?


Omni NLP API

predict_intent: Ajustar_Ponto
score: 0.94
right_intent: Ajustar_Ponto
is it right?: yes
quantity of right answers: 168


Watson Assistent

predict_intent: Ajustar_Ponto
score: 0.93
right_intent: Ajustar_Ponto
is it right?: yes
quantity of right answers: 117


-------------------------------------------
sentence: Como adicionar dependentes ao plano de saúde unimed?


Omni NLP API

predict_intent: Incluir_Dependente_Plano_Saude
score: 0.91
right_intent: Incluir_Dependente_Plano_Saude
is it right?: yes
quantity of right answers: 169


Watson Assistent

predict_intent: Incluir_Dependente_Plano_Saude
score: 0.60
right_intent: Incluir_Dependente_Plano_Saude
is it right?: yes
quantity of right answers: 118


-------------------------------------------
sentence: Quem realiza meus ajustes no ponto?


Omni NLP API

predict_intent: Ajustar_Ponto
score: 0.90
right_intent:

-------------------------------------------
sentence: Gostaria de alterar minhas férias para o fim do ano, como procedo?


Omni NLP API

predict_intent: Agendamento_Alteracao_Ferias
score: 0.93
right_intent: Agendamento_Alteracao_Ferias
is it right?: yes
quantity of right answers: 184


Watson Assistent

predict_intent: Agendamento_Alteracao_Ferias
score: 0.85
right_intent: Agendamento_Alteracao_Ferias
is it right?: yes
quantity of right answers: 131


-------------------------------------------
sentence: Preciso encaminhar o meu veiculo para outro colaborador


Omni NLP API

predict_intent: Transferencia_Veiculo
score: 0.92
right_intent: Transferencia_Veiculo
is it right?: yes
quantity of right answers: 185


Watson Assistent

predict_intent: Transferencia_Veiculo
score: 0.65
right_intent: Transferencia_Veiculo
is it right?: yes
quantity of right answers: 132


-------------------------------------------
sentence: FIZ UMA SOLICITAÇÃO PELO CONCUR DE REEMBOLSO E FOI APROVADO PELO GESTOR